# Hate Speach Detection
I want to build an algorithim to detect hate speach.  This will use the Ethos hate speach dataset at https://www.kaggle.com/datasets/konradb/ethos-hate-speech-dataset.  

In [1]:
import pandas as pd

In [11]:
raw_data = pd.read_csv("Data/labeled", sep = ";", header = 0)
print(binary_data)

                                               comment  isHate
0            You should know women's sports are a joke     1.0
1      You look like Sloth with deeper Down’s syndrome     1.0
2    You look like Russian and speak like Indian. B...     1.0
3                 Women deserve to be abused, I guess.     1.0
4    Women are made for making babies and cooking d...     1.0
..                                                 ...     ...
993   From the midnight sun where the hot springs blow     0.0
994                        Don't say I'm not your type     0.0
995   And therefore never send to know for whom the...     0.0
996                      And I can't stand another day     0.0
997   All values, unless otherwise stated, are in U...     0.0

[998 rows x 2 columns]
                                               comment  violence  \
0            You should know women's sports are a joke  0.000000   
1      You look like Sloth with deeper Down’s syndrome  0.000000   
2    You look li